In [ ]:
%pip install autogen-agentchat~=0.2 langchain langchain-community

In [ ]:
import os
from google.colab import userdata
from langchain.prompts import PromptTemplate
from langchain.agents import Tool
import autogen

In [ ]:
llm_config = {
    "config_list": [{"model": "gpt-4o-mini", "api_key": userdata.get("OPENAI_API_KEY")}],
}

criteria_with_weights = [
    {
        'criterion': "Relevance",
        'weight': 0.25,
        'description': """
        ## 役割
        あなたは採点対象が問題の要求に沿った適切な内容となっているかを評価する役割です。
        ## 評価基準
        * 採点対象が該当箇所の抜き出し、要約、考察などの問題文で求められる指示に正確に従っていること。
        """
    },
    {
        'criterion': "Accuracy",
        'weight': 0.25,
        'description': """
        ## 役割
        あなたは採点対象が解答と比較して正確であるかを評価する役割です。
        ## 評価基準
        * 解答に含まれる重要な情報が漏れなく採点対象に過不足なく記載されていること。
        * 採点対象に誤りが含まれていないこと。
        """
    },
    {
        'criterion': "Expertise",
        'weight': 0.2,
        'description': """
        ## 役割
        あなたは採点対象における用語の使用が適切であること、および用語が文脈に応じて正確な意味で使われていることを評価する役割です。
        ## 評価基準
        * 解答に含まれる専門用語が採点対象においても正確に使用されていること。
        * 採点対象において専門用語が誤解を生むような使われ方をしていないこと。
        """
    },
    {
        'criterion': "Evidence",
        'weight': 0.05,
        'description': """
        ## 役割
        あなたは採点対象が問題に関連する根拠や参照箇所を正確に提示しているかを評価する役割です。
        ## 評価基準
        * 採点対象で提示された根拠が問題と直接関連しており、具体的かつ信頼性の高いものであること。
        * 採点対象に記載された内容の根拠が曖昧でないこと。
        * 問題に関係のない情報が採点対象に含まれていないこと。
        """
    },
    {
        'criterion': "Calculation",
        'weight': 0.1,
        'description': """
        ## 役割
        あなたは採点対象に計算が含まれる場合にその結果の正確性を評価する役割です。
        ## 評価基準
        * 採点対象内の計算結果に誤りがなく正しい結果が導き出されていること。なお、問題が計算を含まない場合には本項目は評価対象外（N/A）として扱う。
        """
    },
    {
        'criterion': "Expression",
        'weight': 0.05,
        'description': """
        ## 役割
        あなたは採点対象が論理的で読みやすく簡潔に記述されているかを評価する役割です。
        ## 評価基準
        * 採点対象に読みにくい表現や冗長な説明が含まれていないこと。
        * 採点対象に解答の要点がわかりやすく整理されていること。
        * 採点対象が論理的な文章であること。
        """
    }
]

# エージェントを基準ごとに生成
agents = list()
for criterion in criteria_with_weights:
    agent = autogen.AssistantAgent(
        name=criterion['criterion'],
        llm_config=llm_config,
        system_message=(
            f"あなたは採点者です。"
            f" この基準の重みは{criterion['weight']}です。以下の説明に従い、公平かつ正確に採点を行ってください。\n\n"
            f"説明: {criterion['description']}"
        ),
    )
    agents.append(agent)

# グループチャットの設定
group_chat = autogen.GroupChat(
    agents=agents, messages=[], max_round=len(agents) + 2
)

# 採点業務管理
manager = autogen.GroupChatManager(
    groupchat=group_chat,
    llm_config=llm_config
)

# 採点基準の注入
# manager.groupchat.context = {"criteria_with_weights": criteria_with_weights}

In [ ]:
def inference(question):
    try:
        # Start the chat with the given question
        manager.initiate_chat(
            message=question,
            recipient=agents[0]
        )
        output = manager.groupchat.messages[-1]["content"]
    except Exception as e:
        output = f"Error: {e}"
    return output

def evaluate_by_multiagent_llm(pred, answer, question=None):
    prompt_template = """
    # 評価基準
    問題と解答を参照しながら採点対象を採点してください。
    各エージェントに適切に採点業務を割り振って、最終的な点数を出力してください。
    例えば、計算箇所がない問題に対して数値計算を評価するエージェントに採点業務を振る必要はないです。
    また、各項目について0〜5点のスコアを付け、理由を添えてください。
    スコアについて、0点は全く正確でない場合、3点は一部正確だが誤りも含む場合、5点は完全に正確である場合とします。

    # 無視事項
    以下の内容については採点不要です。
    - 他の評価基準
    - 問題文や解答の情報

    # 注意点
    データが見つからなかったという文言やエラー文が含まれている文章はスコアを0点にしてください。
    スコアの箇所は必ず【】で囲って明示してください。スコア以外の部分には【】を使用しないでください。
    """

    if question is not None:
        base_prompt = PromptTemplate(
            input_variables=["pred", "answer", "question"],
            template=f"""
            問題: "{{question}}"
            採点対象: "{{pred}}"
            解答: "{{answer}}",
            {prompt_template}
            """
        )
    else:
        base_prompt = PromptTemplate(
            input_variables=["pred", "answer"],
            template=f"""
            採点対象: "{{pred}}"
            解答: "{{answer}}",
            {prompt_template}
            """
        )
    prompt = base_prompt.format(pred=pred, answer=answer, question=question)
    output = inference(prompt)
    return output

In [ ]:
question = """
状況：
あなたは鉄道会社の保守管理責任者です。石勝線での列車脱線事故を受けて、同様の事故を防ぐための点検体制の見直しを任されました。

設問：
省令第87条（施設及び車両の保全）に基づき、減速機の吊りピンについて、どのような点検項目と頻度を設定すべきですか？具体的に3つ以上挙げてください。
点検時に「軽微な摩耗」が見つかった場合、運行継続の判断基準をどのように設定しますか？省令の安全確保の観点から説明してください。
"""

answer = """
点検は3段階で実施することが望ましいです。
まず、毎日の運行前には基本的な日常点検を行います。これには吊りピンの位置確認、ナットの緩みチェック、異音や異常振動の確認が含まれます。
次に、月次の定期点検では、デジタルノギスを使用した吊りピンの摩耗量測定、割りピンの状態確認、ナットの締め付けトルクチェック、そして周辺部品との干渉がないかの確認を行います。
さらに、3ヶ月ごとの精密検査では、非破壊検査による吊りピン内部の亀裂検査、溝付き六角ナットの詳細な摩耗状態検査、減速機取付部全体の応力測定を実施します。
"""

pred = '''
省令第87条に基づく減速機の吊りピンに関する点検項目と頻度は以下の通りです。

1. **点検項目**:
   - 吊りピンの摩耗状態の確認
   - 吊りピンの取り付け状態（緩みや異常の有無）
   - 吊りピン周辺の腐食や損傷の確認

2. **点検頻度**:
   - 定期点検: 1ヶ月ごと
   - 年次点検: 年に1回の詳細な点検

「軽微な摩耗」が見つかった場合の運行継続の判断基準は、以下のように設定します。

- **運行継続の判断基準**:
  - 摩耗の程度が許容範囲内であることを確認するため、具体的な摩耗基準を設定し、その基準を超えない場合は運行を継続する。
  - 摩耗の進行状況を定期的にモニタリングし、次回の点検までに摩耗が進行する可能性がある場合は、運行を一時停止し、早急に点検を行う。

この基準は、省令の安全確保の観点から、運行中の安全を最優先に考慮したものです。
'''
result = evaluate_by_multiagent_llm(pred, answer, question)

chat_manager (to Relevance):


            問題: "
状況：
あなたは鉄道会社の保守管理責任者です。石勝線での列車脱線事故を受けて、同様の事故を防ぐための点検体制の見直しを任されました。

設問：
省令第87条（施設及び車両の保全）に基づき、減速機の吊りピンについて、どのような点検項目と頻度を設定すべきですか？具体的に3つ以上挙げてください。
点検時に「軽微な摩耗」が見つかった場合、運行継続の判断基準をどのように設定しますか？省令の安全確保の観点から説明してください。
"
            採点対象: "
省令第87条に基づく減速機の吊りピンに関する点検項目と頻度は以下の通りです。

1. **点検項目**:
   - 吊りピンの摩耗状態の確認
   - 吊りピンの取り付け状態（緩みや異常の有無）
   - 吊りピン周辺の腐食や損傷の確認

2. **点検頻度**:
   - 定期点検: 1ヶ月ごと
   - 年次点検: 年に1回の詳細な点検

「軽微な摩耗」が見つかった場合の運行継続の判断基準は、以下のように設定します。

- **運行継続の判断基準**:
  - 摩耗の程度が許容範囲内であることを確認するため、具体的な摩耗基準を設定し、その基準を超えない場合は運行を継続する。
  - 摩耗の進行状況を定期的にモニタリングし、次回の点検までに摩耗が進行する可能性がある場合は、運行を一時停止し、早急に点検を行う。

この基準は、省令の安全確保の観点から、運行中の安全を最優先に考慮したものです。
"
            解答: "
点検は3段階で実施することが望ましいです。
まず、毎日の運行前には基本的な日常点検を行います。これには吊りピンの位置確認、ナットの緩みチェック、異音や異常振動の確認が含まれます。
次に、月次の定期点検では、デジタルノギスを使用した吊りピンの摩耗量測定、割りピンの状態確認、ナットの締め付けトルクチェック、そして周辺部品との干渉がないかの確認を行います。
さらに、3ヶ月ごとの精密検査では、非破壊検査による吊りピン内部の亀裂検査、溝付き六角ナットの詳細な摩耗状態検査、減速機取付部全体の応力測定を実施します。
",
            
    # 評価基準
    問題と解答を参照しながら

In [ ]:
print(result)

採点基準に基づいて、'Calculation'に対する評価を行いました。

---

1. **計算結果の正確性**：採点対象には具体的な計算や数値が含まれていないため、計算結果の評価は行うことができません。設問内容も計算を必要としていないため、この点において問題はありません。

2. **問題なしの適切さ**：計算に関する要素が含まれていないことを考えると、これは評価対象外（N/A）として扱われます。したがって、計算に関連する誤りや不正確性は生じていないと判断できます。

結論として、'Calculation'の内容は該当しないため、評価を必要としませんが、全体の評価において計算に関する誤りは認められないため、スコアはN/Aとなります。
